In [ ]:
!echo CUDA AND NVIDIA INSTALLATION
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb;
!dpkg -i cuda-repo-ubuntu1604_8.0.61-1_amd64.deb;
!apt-get update -qq;
!apt-get install cuda-8.0;
!ln -sf /usr/local/cuda-8.0 /usr/local/cuda

import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda/lib'

!apt-get install gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

CUDA AND NVIDIA INSTALLATION
--2022-11-18 05:04:56--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2706 (2.6K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’

cuda-repo-ubuntu160 100%[===================>]   2.64K  --.-KB/s    in 0s      

2022-11-18 05:04:56 (117 MB/s) - ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’ saved [2706/2706]

Selecting previously unselected package cuda-repo-ubuntu1604.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604_8.0.61-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1604 (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604 (8.0.61-1) ...
Note: Check first if ap

In [ ]:
!apt-get install libeigen3-dev;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  libeigen3-doc libmrpt-dev
The following NEW packages will be installed:
  libeigen3-dev
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 810 kB of archives.
After this operation, 7,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libeigen3-dev all 3.3.4-4 [810 kB]
Fetched 810 kB in 1s (796 kB/s)
Selecting previously unselected package libeigen3-dev.
(Reading database ... 134140 files and directories currently installed.)
Preparing to unpack .../libeigen3-dev_3.3.4-4_all.deb ...
Unpacking libeigen3-dev (3.3.4-4) ...
Setting up libeigen3-dev (3.3.4-4) ...


In [ ]:
%%cu
#include <iostream>
#include <eigen3/Eigen/Sparse>
#include <cuda_runtime.h>
#include <algorithm> 
#include <fstream>

#define N 512

using namespace Eigen;

int n_x, n_y, i, j;

double dx, dy, Re;
double velocity = 1.0;

double l2_norm_x = 0.0;
double alpha_uv = 1e-4;
double epsilon_uv = 0.75;

double l2_norm_y = 0.0;

double l2_norm_p = 0.0;
double epsilon_p = 1e-4;
double alpha_p = 0.5;

void link_coefficients(MatrixXd &A_p,MatrixXd &A_e,MatrixXd &A_w,MatrixXd &A_n,MatrixXd &A_s,VectorXd &source_x,VectorXd &source_y,MatrixXd &u_face,MatrixXd &v_face,MatrixXd &p,MatrixXd &u_star,MatrixXd &v_star) {
    int n;
    double D_e,D_w,D_n,D_s,F_e,F_n,F_s,F_w;
    
    D_e=dy/(dx*Re);
    D_w=dy/(dx*Re);
    D_n=dx/(dy*Re);
    D_s=dx/(dy*Re);
    
    for(i=2;i<n_y;i++)
    {
        for (j=2;j<n_x;j++)
        {
            n=(i-1)*n_x + (j-1);

            F_e=dy*u_face(i,j);
            F_w=dy*u_face(i,j-1);
            F_n=dx*v_face(i-1,j);
            F_s=dx*v_face(i,j);

            A_e(i,j)=D_e + std::max(0.0,-F_e);
            A_w(i,j)=D_w + std::max(0.0,F_w);
            A_n(i,j)=D_n + std::max(0.0,-F_n);
            A_s(i,j)=D_s + std::max(0.0,F_s);
            A_p(i,j)=D_e + D_w + D_n + D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n);

            source_x[n]=0.5*alpha_uv*(p(i,j-1)-p(i,j+1))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j);
            source_y[n]=0.5*alpha_uv*(p(i+1,j)-p(i-1,j))*dx + (1-alpha_uv)*A_p(i,j)*v_star(i,j);
        }
    }

    j=1;
    for (i=2;i<n_y;i++)
    { 
        n=(i-1)*n_x ;

        F_e=dy*u_face(i,j);
        F_w=dy*u_face(i,j - 1);
        F_n=dx*v_face(i - 1,j);
        F_s=dx*v_face(i,j);

        A_e(i,j)=D_e + std::max(0.0,-F_e);
        A_n(i,j)=D_n + std::max(0.0,-F_n);
        A_s(i,j)=D_s + std::max(0.0,F_s);
        A_p(i,j)=D_e + 2*D_w + D_n + D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n);

        source_x[n]=0.5*alpha_uv*(p(i,j) - p(i,j + 1))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j);  // P_o - 0.5(P_o+P_e)
        source_y[n]=0.5*alpha_uv*(p(i + 1,j) - p(i - 1,j))*dx + (1-alpha_uv)*A_p(i,j)*v_star(i,j);
    }

    i=n_y;
    for(j=2;j<n_x;j++)
    {
        n=(n_y-1)*n_x + (j-1);

        F_e=dy*u_face(i,j);
        F_w=dy*u_face(i,j - 1);
        F_n=dx*v_face(i - 1,j);
        F_s=dx*v_face(i,j);

        A_e(i,j)=D_e + std::max(0.0,-F_e);
        A_w(i,j)=D_w + std::max(0.0,F_w);
        A_n(i,j)=D_n + std::max(0.0,-F_n);
        A_p(i,j)=D_e + D_w + D_n + 2*D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n);

        source_x[n]=0.5*alpha_uv*(p(i,j - 1) - p(i,j + 1))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j);
        source_y[n]=0.5*alpha_uv*(p(i,j) - p(i - 1,j))*dx + (1-alpha_uv)*A_p(i,j)*v_star(i,j);
    }

    j=n_x;
    for(i=2;i<n_y;i++)
    {
        n=i*n_x -1;    
     
        F_e=dy*u_face(i,j);
        F_w=dy*u_face(i,j - 1);
        F_n=dx*v_face(i - 1,j);
        F_s=dx*v_face(i,j);

        A_w(i,j)=D_w + std::max(0.0,F_w);
        A_n(i,j)=D_n + std::max(0.0,-F_n);
        A_s(i,j)=D_s + std::max(0.0,F_s);
        A_p(i,j)=2*D_e + D_w + D_n + D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n);

        source_x[n]=0.5*alpha_uv*(p(i,j-1) - p(i,j))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j);
        source_y[n]=0.5*alpha_uv*(p(i + 1,j) - p(i - 1,j))*dx + (1-alpha_uv)*A_p(i,j)*v_star(i,j);
    }

    i=1;
    for(j=2;j<n_x;j++)
    {
        n=(j-1);

        F_e=dy*u_face(i,j);
        F_w=dy*u_face(i,j - 1);
        F_n=dx*v_face(i - 1,j);
        F_s=dx*v_face(i,j);

        A_e(i,j)=D_e + std::max(0.0,-F_e);
        A_w(i,j)=D_w + std::max(0.0,F_w);
        A_s(i,j)=D_s + std::max(0.0,F_s);
        A_p(i,j)=D_e + D_w + 2*D_n + D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n) ;

        source_x[n]=0.5*alpha_uv*(p(i,j - 1) - p(i,j + 1))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j) + alpha_uv*velocity*(2*D_n + std::max(0.0,-F_n));
        source_y[n]=0.5*alpha_uv*(p(i + 1,j) - p(i,j))*dx  + (1-alpha_uv)*A_p(i,j)*v_star(i,j);
    }

    i=1;
    j=1;
    n=0;

    F_e=dy*u_face(i,j);
    F_w=dy*u_face(i,j - 1);
    F_n=dx*v_face(i - 1,j);
    F_s=dx*v_face(i,j);

    A_e(i,j)=D_e + std::max(0.0,-F_e);
    A_s(i,j)=D_s + std::max(0.0,F_s);
    A_p(i,j)=D_e + 2*D_w + 2*D_n + D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n) ;

    source_x[n]=0.5*alpha_uv*(p(i,j) - p(i,j + 1))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j) + alpha_uv*velocity*(2*D_n + std::max(0.0,-F_n));
    source_y[n]=0.5*alpha_uv*(p(i + 1,j) - p(i,j))*dx + (1-alpha_uv)*A_p(i,j)*v_star(i,j);

    i=1;
    j=n_x;
    n=n_x-1;

    F_e=dy*u_face(i,j);
    F_w=dy*u_face(i,j - 1);
    F_n=dx*v_face(i - 1,j);
    F_s=dx*v_face(i,j);

    A_w(i,j)=D_w + std::max(0.0,F_w);
    A_s(i,j)=D_s + std::max(0.0,F_s);
    A_p(i,j)=2*D_e + D_w + 2*D_n + D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n);

    source_x[n]=0.5*alpha_uv*(p(i,j - 1) - p(i,j))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j) + alpha_uv*velocity*(2*D_n + std::max(0.0,-F_n));
    source_y[n]=0.5*alpha_uv*(p(i + 1,j) - p(i,j))*dx + (1-alpha_uv)*A_p(i,j)*v_star(i,j);

    i=n_y;
    j=1;
    n=(n_y-1)*n_x ;


    F_e=dy*u_face(i,j);
    F_w=dy*u_face(i,j - 1);
    F_n=dx*v_face(i - 1,j);
    F_s=dx*v_face(i,j);

    A_e(i,j)=D_e + std::max(0.0,-F_e);
    A_n(i,j)=D_n + std::max(0.0,-F_n);
    A_p(i,j)=D_e + 2*D_w + D_n + 2*D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n);

    source_x[n]=0.5*alpha_uv*(p(i,j) - p(i,j + 1))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j);  //P_o - 0.5(P_o+P_e)
    source_y[n]=0.5*alpha_uv*(p(i,j) - p(i - 1,j))*dx + (1-alpha_uv)*A_p(i,j)*v_star(i,j);  //P_o - 0.5(P_o+P_n)

    i=n_y;
    j=n_x;
    n=n_x*n_y-1;

    F_e=dy*u_face(i,j);
    F_w=dy*u_face(i,j - 1);
    F_n=dx*v_face(i - 1,j);
    F_s=dx*v_face(i,j);

    A_w(i,j)=D_w + std::max(0.0,F_w);
    A_n(i,j)=D_n + std::max(0.0,-F_n);
    A_p(i,j)=2*D_e + D_w + D_n + 2*D_s + std::max(0.0,F_e) + std::max(0.0,-F_w) + std::max(0.0,-F_s) + std::max(0.0,F_n);

    source_x[n]=0.5*alpha_uv*(p(i,j - 1)- p(i,j))*dy + (1-alpha_uv)*A_p(i,j)*u_star(i,j);  //0.5(P_w+P_o)-P_o
    source_y[n]=0.5*alpha_uv*(p(i,j) - p(i-1,j))*dx  + (1-alpha_uv)*A_p(i,j)*v_star(i,j); //P_o - 0.5(P_o+P_n)


    A_e=alpha_uv*A_e;
    A_w=alpha_uv*A_w;
    A_n=alpha_uv*A_n;
    A_s=alpha_uv*A_s;



}

void build_coefficient_matrix(SparseMatrix <double> &A,MatrixXd &ap,MatrixXd &ae,MatrixXd &aw,MatrixXd &an,MatrixXd &as){


   typedef Triplet<double> T;
   std::vector<T> tripletList;
   
   int n;

   int i=1;
   for (int j=2;j<n_x;j++)
   {
        n=(j-1);

        tripletList.push_back(T(n,n,ap(i,j)));
        tripletList.push_back(T(n,n-1,-aw(i,j)));
        tripletList.push_back(T(n,n+1,-ae(i,j)));
        tripletList.push_back(T(n,n+n_x,-as(i,j)));

   }
   
   for(int i=2;i<n_y;i++)
   {
       for(int j=2;j<n_x;j++)
       {
           n=(i-1)*n_x + (j-1);

           tripletList.push_back(T(n,n,ap(i,j)));
           tripletList.push_back(T(n,n-1,-aw(i,j)));
           tripletList.push_back(T(n,n+1,-ae(i,j)));
           tripletList.push_back(T(n,n-n_x,-an(i,j)));
           tripletList.push_back(T(n,n+n_x,-as(i,j)));

           
        }
    }
    
    int j=1;
    for(int i=2;i<n_y;i++)
    {
           n=(i-1)*n_x ;

           tripletList.push_back(T(n,n,ap(i,j)));
           tripletList.push_back(T(n,n+1,-ae(i,j)));
           tripletList.push_back(T(n,n-n_x,-an(i,j)));
           tripletList.push_back(T(n,n+n_x,-as(i,j)));

    }
    
    
    j=n_x;
    for(int i=2;i<n_y;i++)
    {
        n=i*n_x -1;

        tripletList.push_back(T(n,n,ap(i,j)));
        tripletList.push_back(T(n,n-1,-aw(i,j)));
        tripletList.push_back(T(n,n-n_x,-an(i,j)));
        tripletList.push_back(T(n,n+n_x,-as(i,j)));   
    }
   
   i =n_y;
   for (int j=2;j<n_x;j++)
   {
       n=(n_y-1)*n_x + (j-1);

       tripletList.push_back(T(n,n,ap(i,j)));
       tripletList.push_back(T(n,n-1,-aw(i,j)));
       tripletList.push_back(T(n,n+1,-ae(i,j)));
       tripletList.push_back(T(n,n-n_x,-an(i,j)));

   }
    
    n=0;
    i=1;
    j=1;
    tripletList.push_back(T(n,n,ap(i,j)));
    tripletList.push_back(T(n,n+1,-ae(i,j)));
    tripletList.push_back(T(n,n+n_x,-as(i,j)));
     
    
    i=1;
    j=n_x;
    n=n_x-1;

    tripletList.push_back(T(n,n,ap(i,j)));
    tripletList.push_back(T(n,n-1,-aw(i,j)));
    tripletList.push_back(T(n,n+n_x,-as(i,j)));
    
    n=(n_y-1)*n_x ;
    j=1;
    i=n_y;

    tripletList.push_back(T(n,n,ap(i,j)));
    tripletList.push_back(T(n,n+1,-ae(i,j)));
    tripletList.push_back(T(n,n-n_x,-an(i,j)));
    
    n=(n_x*n_y)-1;
    i=n_y;
    j=n_x;

    tripletList.push_back(T(n,n,ap(i,j)));
    tripletList.push_back(T(n,n-1,-aw(i,j)));
    tripletList.push_back(T(n,n-n_x,-an(i,j)));
    
    
    A.setFromTriplets(tripletList.begin(),tripletList.end());

}

void solve(SparseMatrix <double> &A,VectorXd &b,MatrixXd &phi,double &l2_norm,double &epsilon)
{
    VectorXd x(n_x*n_y);

    int n=0;
    for(int i=1;i<n_y+1;i++)
    {
        for (int j=1;j<n_x+1;j++)
        {
         
         x[n]=phi(i,j);
         n+=1;

        }
    }
    l2_norm=(A*x-b).norm() ;

    BiCGSTAB<SparseMatrix<double> > solver;
    solver.compute(A);
    solver.setTolerance(epsilon);
    x = solver.solve(b);

    n=0;
    for(int i=1;i<n_y+1;i++)
    {
        for (int j=1;j<n_x+1;j++)
        {
         
         phi(i,j)=x[n];
         n+=1;

        }
    }
}

__global__ void face_velocity(MatrixXd *u, MatrixXd *v, MatrixXd *u_face, MatrixXd *v_face, MatrixXd *p, MatrixXd *A_p, int n_x, int n_y, double alpha_uv) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    double dy = 1.0 / n_y;
    
    for(int i = 1; i < n_y + 1; i++) {
      for (int j = 1; j < n_x; j++) {
        u_face(i,j)[index] = 0.5 * (u(i,j)[index] + u(i,j + 1)[index]) + 0.25 * alpha_uv * (p(i,j + 1)[index] - p(i,j - 1)[index]) * dy / A_p(i,j)[index] + 0.25 * alpha_uv * (p(i,j + 2)[index] - p(i,j)[blockIdx.x]) * dy / A_p(i,j + 1)[index] - 0.5 * alpha_uv * (1 / A_p(i,j)[index] + 1 / A_p(i,j + 1)[index]) * (p(i,j + 1)[index] - p(i,j)[index]) * dy;         
      }  
    }


    for(int i = 2; i < n_y + 1; i++) {
        for (int j = 1; j < n_x + 1; j++) {
            v_face(i-1,j)[index] = 0.5 * (v(i,j)[index] + v(i - 1,j)[index]) + 0.25 * alpha_uv * (p(i - 1,j)[index] - p(i + 1,j)[index]) * dy / A_p(i,j)[index] + 0.25 * alpha_uv * (p(i - 2,j)[index] - p(i,j)[blockIdx.x]) * dy / A_p(i - 1,j)[index]- 0.5 * alpha_uv * (1 / A_p(i,j)[index] + 1 / A_p(i - 1,j)[index]) * (p(i - 1,j)[index] - p(i,j)[index]) * dy;
        }
    }
}

void pressure_correction_link_coefficients(MatrixXd &u,MatrixXd &u_face,MatrixXd &v_face,MatrixXd &Ap_p,MatrixXd &Ap_e,MatrixXd &Ap_w,MatrixXd &Ap_n,MatrixXd &Ap_s,VectorXd &source_p,MatrixXd &A_p) {
    int n;
    for(i=2;i<n_y;i++)
    {
        for(j=2;j<n_x;j++)
        {
            n=(i-1)*n_x + (j-1);

            Ap_e(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j + 1))*(dy*dy);
            Ap_w(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j - 1))*(dy*dy);
            Ap_n(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i - 1,j))*(dx*dx);
            Ap_s(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i + 1,j))*(dx*dx);
            Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

            source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;

        } 

    }
    
    i=1;
    for(j=2;j<n_x;j++)
    { 
        n=(j-1);
    
        Ap_e(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j + 1))*(dy*dy);
        Ap_w(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j - 1))*(dy*dy);
        Ap_s(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i + 1,j))*(dx*dx);
        Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

        source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;
    }
    
    j=1;
    for(i=2;i<n_y;i++)
    { 
        n=(i-1)*n_x ;

        Ap_e(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j + 1))*(dy*dy);
        Ap_n(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i - 1,j))*(dx*dx);
        Ap_s(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i + 1,j))*(dx*dx);
        Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

        source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;
    }

    j=n_x;
    for(i=2;i<n_y;i++)
    {
        n=i*n_x -1;    

        Ap_w(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j - 1))*(dy*dy);
        Ap_n(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i - 1,j))*(dx*dx);
        Ap_s(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i + 1,j))*(dx*dx);
        Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

        source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;
    }

    i=n_y;
    for(j=2;j<n_x;j++)
    { 
        n=(n_y-1)*n_x + (j-1);

        Ap_e(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j + 1))*(dy*dy);
        Ap_w(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j - 1))*(dy*dy);
        Ap_n(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i - 1,j))*(dx*dx);
        Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

        source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;
    }

    i=1;
    j=1;
    n=0;

    Ap_e(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j + 1))*(dy*dy);
    Ap_s(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i + 1,j))*(dx*dx);
    Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

    source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;
    
    i=1;
    j=n_x;
    n=n_x-1;

    Ap_w(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j - 1))*(dy*dy);
    Ap_s(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i + 1,j))*(dx*dx);
    Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

    source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;
    
    i=n_y;
    j=1;
    n=(n_y-1)*n_x ;

    Ap_e(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j + 1))*(dy*dy);
    Ap_n(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i - 1,j))*(dx*dx);
    Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

    source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;
    
    i=n_y;
    j=n_x;
    n=n_x*n_y-1;

    Ap_w(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i,j - 1))*(dy*dy);
    Ap_n(i,j)=0.5*alpha_uv*(1/A_p(i,j) + 1/A_p(i - 1,j))*(dx*dx);
    Ap_p(i,j)=Ap_e(i,j) + Ap_w(i,j) + Ap_n(i,j) + Ap_s(i,j);

    source_p[n]=-(u_face(i,j) - u_face(i,j - 1))*dy - (v_face(i - 1,j) - v_face(i,j))*dx;   
}

void correct_pressure(MatrixXd &p_star,MatrixXd &p,MatrixXd &p_prime)
{
    
    p_prime.block(0,1,1,n_x)=p_prime.block(1,1,1,n_x);
    
    p_prime.block(1,0,n_y,1)=p_prime.block(1,1,n_y,1);
    
    p_prime.block(1,n_x+1,n_y,1)=p_prime.block(1,n_x,n_y,1);
    
    p_prime.block(n_y+1,1,1,n_x)=p_prime.block(n_y,1,1,n_x);
    
    p_prime(0,0)=(p_prime(1,1)+p_prime(0,1)+p_prime(1,0))/3;

    p_prime(0,n_x+1)=(p_prime(0,n_x)+p_prime(1,n_x)+p_prime(1,n_x+1))/3;

    p_prime(n_y+1,0)=(p_prime(n_y,0)+p_prime(n_y,1)+p_prime(n_y+1,1))/3;

    p_prime(n_y+1,n_x+1)=(p_prime(n_y,n_x+1)+p_prime(n_y+1,n_x)+p_prime(n_y,n_x))/3;

    MatrixXd p_ref=MatrixXd::Constant(n_y+2,n_x+2,p_prime(0,0));
    
    
    p_star=p+alpha_p*(p_prime);
}

__global__ void correct_cell_center_velocity(MatrixXd *u, MatrixXd *v, MatrixXd *u_star, MatrixXd *v_star, MatrixXd *p_prime, MatrixXd *A_p, int n_x, int n_y, double alpha_uv) {
    int i, j;
    double dx = 1.0 / n_x;
    double dy = 1.0 / n_y;

    for(i = 1; i < n_y + 1; i++) {
        for(j = 2; j < n_x; j++) {
            u_star(i,j)[blockIdx.x] = u(i,j)[blockIdx.x] + 0.5 * alpha_uv * (p_prime(i,j-1)[blockIdx.x] - p_prime(i,j+1)[blockIdx.x]) * dy / A_p(i,j)[blockIdx.x];
        }
    }

    j = 1;
    for(i = 1; i < n_y + 1; i++) {
        u_star(i,j)[blockIdx.x] = u(i,j)[blockIdx.x] + 0.5 * alpha_uv * (p_prime(i,j)[blockIdx.x] - p_prime(i,j+1)[blockIdx.x]) * dy / A_p(i,j)[blockIdx.x];
    }

    j = n_x;
    for(i = 1; i < n_y + 1; i++) {
        u_star(i,j)[blockIdx.x] = u(i,j)[blockIdx.x] + 0.5 * alpha_uv * (p_prime(i,j-1)[blockIdx.x] - p_prime(i,j)[blockIdx.x]) * dy / A_p(i,j)[blockIdx.x];
    }

    for(i = 2; i < n_y; i++) {
        for(j = 1; j < n_x + 1; j++) {
            v_star(i,j)[blockIdx.x] = v(i,j)[blockIdx.x] + 0.5 * alpha_uv * (p_prime(i+1,j)[blockIdx.x] - p_prime(i-1,j)[blockIdx.x]) * dx / A_p(i,j)[blockIdx.x];
        }
    }

    i = 1;
    for(j = 1; j < n_x + 1; j++) { 
        v_star(i,j)[blockIdx.x] = v(i,j)[blockIdx.x] + 0.5 * alpha_uv * (p_prime(i + 1,j)[blockIdx.x] - p_prime(i,j)[blockIdx.x]) * dx / A_p(i,j)[blockIdx.x];
    }

    i = n_y;
    for(j = 1; j < n_x + 1; j++) {
        v_star(i,j)[blockIdx.x] = v(i,j)[blockIdx.x] + 0.5 * alpha_uv * (p_prime(i,j)[blockIdx.x] - p_prime(i - 1,j)[blockIdx.x]) * dx / A_p(i,j)[blockIdx.x];
    }
}

__global__ void correct_face_velocity(MatrixXd *u_face, MatrixXd *v_face, MatrixXd *p_prime, MatrixXd *A_p, int n_x, int n_y, double alpha_uv)
{
    double dx = 1.0 / n_x;
    double dy = 1.0 / n_y;
 
    for(int i = 1; i < n_y + 1; i++) { 
        for(int j = 1; j < n_x; j++) {
            u_face(i,j)[blockIdx.x] = u_face(i,j)[blockIdx.x] + 0.5 * alpha_uv * (1 / A_p(i,j)[blockIdx.x] + 1 / A_p(i,j+1)[blockIdx.x]) * (p_prime(i,j) - p_prime(i,j+1)[blockIdx.x]) * dy;
        }
    }

    for(int i = 2; i < n_y + 1; i++) {
        for(int j = 1; j <n_x + 1; j++) {
            v_face(i-1,j)[blockIdx.x] = v_face(i-1,j)[blockIdx.x] + 0.5 * alpha_uv * (1 / A_p(i,j)[blockIdx.x] + 1 / A_p(i-1,j)[blockIdx.x]) * (p_prime(i,j)[blockIdx.x] - p_prime(i-1,j)[blockIdx.x]) * dx;
        }
    }
}

void post_processing(MatrixXd u_star,MatrixXd v_star,MatrixXd p_star)
{
    VectorXd x(n_x+2),y(n_y+2);
    x << 0,VectorXd::LinSpaced(n_x,dx/2.0,1-dx/2.0),1;
    y << 0,VectorXd::LinSpaced(n_y,dy/2.0,1-dy/2.0),1;

     std::ofstream outFile;
     outFile.open("./u.dat");
     outFile << u_star;
     outFile.close();

     outFile.open("./v.dat");
     outFile << v_star;
     outFile.close();

     outFile.open("./p.dat");
     outFile << p_star;
     outFile.close();

     outFile.open("./x.dat");
     outFile << x;
     outFile.close();

     outFile.open("./y.dat");
     outFile << y;
     outFile.close();
}

int main () {
    n_x = 35;
    n_y = 35;
    
    dx=1.0/n_x;
    dy=1.0/n_y;
    
    Re = 100;
 
    // Declaracion de variables primitivas
    MatrixXd u(n_y + 2, n_x + 2), u_star(n_y + 2, n_x + 2);
    MatrixXd v(n_y + 2, n_x + 2), v_star(n_y + 2, n_x + 2);
    MatrixXd p(n_y + 2, n_x + 2), p_star(n_y + 2, n_x + 2), p_prime(n_y + 2, n_x + 2);

    // Declarando condiciones de momentum
    MatrixXd A_p(n_y + 2, n_x + 2), A_e(n_y + 2, n_x + 2), A_w(n_y + 2, n_x + 2), A_n(n_y + 2, n_x + 2), A_s(n_y + 2, n_x + 2);

    // Declarando coeficiones de correccion de presion
    MatrixXd Ap_p(n_y + 2, n_x + 2), Ap_e(n_y + 2, n_x + 2), Ap_w(n_y + 2, n_x + 2), Ap_n(n_y + 2, n_x + 2), Ap_s(n_y + 2, n_x + 2);

    // Declarando la fuente
    VectorXd source_x(n_y * n_x), source_y(n_y * n_x), source_p(n_y * n_x);

    // Declaracion de velocidades
    MatrixXd u_face(n_y + 2, n_x + 1), v_face(n_y + 1, n_x + 2);
    
    // Condiciones de la matriz y vectores en Ax=b
    SparseMatrix<double> A_momentum(n_x*n_y,n_x*n_y),A_pressure_correction(n_x*n_y,n_x*n_y);

    MatrixXd *dev_u;
    MatrixXd *dev_u_star;
    MatrixXd *dev_v; 
    MatrixXd *dev_v_star;
    MatrixXd *dev_p; 
    MatrixXd *dev_p_prime;
    MatrixXd *dev_A_p;
    MatrixXd *dev_u_face; 
    MatrixXd *dev_v_face;
 
    MatrixXd *host_u = &u;
    MatrixXd *host_u_star = &u_star;
    MatrixXd *host_v = &v; 
    MatrixXd *host_v_star = &v_star;
    MatrixXd *host_p = &p; 
    MatrixXd *host_p_prime = &p_prime;
    MatrixXd *host_A_p = &A_p;
    MatrixXd *host_u_face = &u_face; 
    MatrixXd *host_v_face = &v_face;

    cudaMalloc((void **)&dev_u, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_v, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_u_star, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_v_star, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_p, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_p_prime, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_A_p, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_u_face, sizeof(MatrixXd)*N);
    cudaMalloc((void **)&dev_v_face, sizeof(MatrixXd)*N);
    
    // Condiciones de frontera
    u.block(0,1,1,n_x)=MatrixXd::Constant(1,n_x,1.0);
    u_star.block(0,1,1,n_x)=MatrixXd::Constant(1,n_x,1.0);
    u_face.block(0,1,1,n_x-1)=MatrixXd::Constant(1,n_x-1,1.0);
 
    cudaMemcpy(dev_u, host_u, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_v, host_v, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_u_star, host_u_star, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_v_star, host_v_star, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_p, host_p, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_p_prime, host_p_prime, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_A_p, host_A_p, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_u_face, host_u_face, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_v_face, host_v_face, sizeof(MatrixXd)*N, cudaMemcpyHostToDevice);
    

    int max_outer_iterations=1000;
    
    for(int n=1;n<=max_outer_iterations;n++)
    {
        link_coefficients(A_p,A_e,A_w,A_n,A_s,source_x,source_y,u_face,v_face,p,u_star,v_star);
    
        build_coefficient_matrix(A_momentum,A_p,A_e,A_w,A_n,A_s);
      
        solve(A_momentum,source_x,u,l2_norm_x,epsilon_uv);
        
        solve(A_momentum,source_y,v,l2_norm_y,epsilon_uv);
        
        // Kernel
        face_velocity<<<N,1>>>(dev_u, dev_v, dev_u_face, dev_v_face, dev_p, dev_A_p, n_x, n_y, alpha_uv);
        cudaMemcpy(host_u_face, dev_u_face, sizeof(MatrixXd)*N, cudaMemcpyDeviceToHost);
        cudaMemcpy(host_v_face, dev_v_face, sizeof(MatrixXd)*N, cudaMemcpyDeviceToHost);
        
        pressure_correction_link_coefficients(u,u_face,v_face,Ap_p,Ap_e,Ap_w,Ap_n,Ap_s,source_p,A_p);

        build_coefficient_matrix(A_pressure_correction,Ap_p,Ap_e,Ap_w,Ap_n,Ap_s);

        solve(A_pressure_correction,source_p,p_prime,l2_norm_p,epsilon_p);
        
        correct_pressure(p_star,p,p_prime);

        // Kernel
        correct_cell_center_velocity<<<N,1>>>(dev_u, dev_v, dev_u_star, dev_v_star, dev_p_prime, dev_A_p, n_x, n_y, alpha_uv);
        cudaMemcpy(host_u_star, dev_u_star, sizeof(MatrixXd)*N, cudaMemcpyDeviceToHost);
        cudaMemcpy(host_v_star, dev_v_star, sizeof(MatrixXd)*N, cudaMemcpyDeviceToHost);

        // Kernel
        correct_face_velocity<<<N,1>>>(dev_u_face, dev_v_face, dev_p_prime, dev_A_p, n_x, n_y, alpha_uv);
        cudaMemcpy(host_u_face, dev_u_face, sizeof(MatrixXd)*N, cudaMemcpyDeviceToHost);
        cudaMemcpy(host_v_face, dev_v_face, sizeof(MatrixXd)*N, cudaMemcpyDeviceToHost);
     
        cudaFree(dev_u);
        cudaFree(dev_v);
        cudaFree(dev_u_star);
        cudaFree(dev_u_star);
        cudaFree(dev_p);
        cudaFree(dev_p_prime);
        cudaFree(dev_A_p);
        cudaFree(dev_u_face);
        cudaFree(dev_v_face);

        p=p_star;
    
        std::cout << n << " " << l2_norm_x << " " << l2_norm_y << " " << l2_norm_p << "\n";
        
        if(l2_norm_x < 1e-5 & l2_norm_y < 1e-5 & l2_norm_p < 1e-6)
        { 
        std::cout << "Converge!";
        break;
        }

    }

    post_processing(u_star,v_star,p_star);

}


/tmp/tmpypaazn_7/3338a5d3-dbfb-4e9f-a6ee-e8a58b6be87b.cu(358): error: expression preceding parentheses of apparent call must have (pointer-to-) function type

/tmp/tmpypaazn_7/3338a5d3-dbfb-4e9f-a6ee-e8a58b6be87b.cu(358): error: expression preceding parentheses of apparent call must have (pointer-to-) function type

/tmp/tmpypaazn_7/3338a5d3-dbfb-4e9f-a6ee-e8a58b6be87b.cu(358): error: expression preceding parentheses of apparent call must have (pointer-to-) function type

/tmp/tmpypaazn_7/3338a5d3-dbfb-4e9f-a6ee-e8a58b6be87b.cu(358): error: expression preceding parentheses of apparent call must have (pointer-to-) function type

/tmp/tmpypaazn_7/3338a5d3-dbfb-4e9f-a6ee-e8a58b6be87b.cu(358): error: expression preceding parentheses of apparent call must have (pointer-to-) function type

/tmp/tmpypaazn_7/3338a5d3-dbfb-4e9f-a6ee-e8a58b6be87b.cu(358): error: expression preceding parentheses of apparent call must have (pointer-to-) function type

/tmp/tmpypaazn_7/3338a5d3-dbfb-4e9f-a6ee-e8a58